# Checkpoint 활용


In [2]:
import sys
import requests
import numpy as np
from konlpy.tag import Mecab
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

### 마지막 체크포인트 불러오기

In [ ]:
checkpoint_path =  "D:\\tensorflow_checkpoint\\cp-0002-ckpt" # 나는 이 주소 아닐껄?
restore_model = load_model(checkpoint_path)
restore_model

In [3]:
# 불용어 목록 가져오기

r = requests.get("https://data.hossam.kr/korean_stopwords.txt")
r.encoding = 'utf-8'
stopwords = r.text.split('\n')
print(stopwords)                 

['이', '있', '하', '것', '들', '그', '되', '수', '이', '보', '않', '없', '나', '사람', '주', '아니', '등', '같', '우리', '때', '년', '가', '한', '지', '대하', '오', '말', '일', '그렇', '위하', '때문', '그것', '두', '말하', '알', '그러나', '받', '못하', '일', '그런', '또', '문제', '더', '사회', '많', '그리고', '좋', '크', '따르', '중', '나오', '가지', '씨', '시키', '만들', '지금', '생각하', '그러', '속', '하나', '집', '살', '모르', '적', '월', '데', '자신', '안', '어떤', '내', '내', '경우', '명', '생각', '시간', '그녀', '다시', '이런', '앞', '보이', '번', '나', '다른', '어떻', '여자', '개', '전', '들', '사실', '이렇', '점', '싶', '말', '정도', '좀', '원', '잘', '통하', '소리', '놓', '!', '"', '$', '%', '&', "'", '(', ')', '*', '+', ',', '-', '.', '...', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', ';', '<', '=', '>', '?', '@', '\\', '^', '_', '`', '|', '~', '·', '—', '——', '‘', '’', '“', '”', '…', '、', '。', '〈', '〉', '《', '》', '가', '가까스로', '가령', '각', '각각', '각자', '각종', '갖고말하자면', '같다', '같이', '개의치않고', '거니와', '거바', '거의', '것', '것과 같이', '것들', '게다가', '게우다', '겨우', '견지에서', '결과에 이르다', '결국', '결론을 낼 수 있다', '겸사겸사', '고려하면', '고로', '곧', '공

### 모델 개발 과정에서 얻어진 데이터 크기 값

In [4]:
vocab_size =19204
max_word_count = 512
category = ['IT_과학','경제','사회', '생활_문화', '세계', '정치']

### 임의의 문장에 대한 분류 구현

In [ ]:
# 임의의 문장
comment = input('[아시안게임] 황선홍호, 쿠웨이트 상대 첫 출격…3회 연속 금메달 도전!')

# 형태소 분석 엔진
if sys.platform == 'win32' :
    mecab = Mecab(dicpath='C:\\mecab\\mecab-ko-dic')
else :
    mecab = Mecab()

# 형태소 분석
morphs = mecab.morphs(comment)

# 불용어 제거
temp_word = []
for j in morphs:
    if j not in stopwords:
        temp_word.append(j)

# 입력 데이터의 차원을 맞추기 위해 리스트로 묶음
word_set = [temp_word]

# 자주 등장하는 단어를 제외한 나머지 단어를 OOV로 처리하여 최종 토큰화 수행
# oov_token은 out-of-vocabulary(단어 집합에 없는 단어)

tokenizer = Tokenizer(vocab_size, oov_token = 'OOV')
tokenizer.fit_on_texts(word_set)
token_set = tokenizer.texts_to_sequences(word_set)

# 최대 길이에 맞춰서 패딩 처리
pad_token_set = pad_sequences(token_set, maxlen=max_word_count)

# 전처리가 완료된 말뭉치를 학습모델에 적용하여 예측하기
result = restore_model.predict(pad_token_set)

# 결과 분석
arg_result = np.argmax(result, axis=-1)
v = arg_result[0]

print("%d, %s >> %s" % (v, category[v], comment))

In [ ]:
# git test.py


import sys
import requests
import numpy as np
from konlpy.tag import Mecab
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences


def text_classification(comment):
    r = requests.get("https://data.hossam.kr/korean_stopwords.txt")
    r.encoding = 'utf-8'
    stopwords = r.text.split("\n")

    vocab_size = 19204
    max_word_count = 512
    category = ['IT,과학', '경제', '사회', '생활,문화', '세계', '정치']

    checkpoint_path = "D:\\tensorflow_checkpoint\\cp-0002-ckpt"
    restore_model = load_model(checkpoint_path)

    # 형태소 분석 엔진
    if sys.platform == 'win32':
        mecab = Mecab(dicpath="C:\\mecab\\mecab-ko-dic")
    else:
        mecab = Mecab()

    # 형태소 분석
    morphs = mecab.morphs(comment)

    # 불용어 제거
    tmp_word = []
    for j in morphs:
        if j not in stopwords:
            tmp_word.append(j)

    # 입력 데이터의 차원을 맞추기 위해 리스트로 감쌈
    word_set = [tmp_word]

    # 자주 등장하는 단어를 제외한 나머지 단어를 OOV로 처리하여 최종 토큰화 수행
    tokenizer = Tokenizer(vocab_size, oov_token = 'OOV')
    tokenizer.fit_on_texts(word_set)
    token_set = tokenizer.texts_to_sequences(word_set)

    # 최대 길이에 맞춰서 패딩 처리
    pad_token_set = pad_sequences(token_set, maxlen=max_word_count)

    # 전처리가 완료된 말뭉치를 학습모델에 적용하여 예측하기
    result = restore_model.predict(pad_token_set)

    # 결과 분석
    arg_result = np.argmax(result, axis=-1)
    v = arg_result[0]

    return category[v]


# 터미널을 통해 직접 실행하는 경우만 구동되는 구문
if __name__ == "__main__": 
    args = sys.argv[1:]
    comment = " ".join(args)
    
    category = text_classification(comment)
    
    print("입력한 문장의 카테고리는 {}입니다.".format(category))

arges = sys.argv[1:] 
파일 이름을 제외한 나머지

띄어쓰기를 args에 join
그래서 comment는 하나의 완전한 문장이 된다. 